# mAP Implementation with details
- We will implement Mean Average Precision (mAP) with pretrained YOLOv3 model (which publicly available on pytorch model)

In [25]:
import torch
import numpy as np

# output = torch.zeros(2, 30, 7, 7)
output = torch.unsqueeze(torch.unsqueeze(torch.from_numpy(np.array([
    # x, y, w, h, c, CIDs
    0.5, 0.5, 1.0, 1.0, 1.0,
    0.5, 0.5, 1.0, 1.0, 0.1,
    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
    0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
])).repeat(2, 1), 2), 3).repeat(1, 1, 7, 7).float()
label = torch.zeros(2, 25, 7, 7)

print("output", output.shape)
print("label", label.shape)

output torch.Size([2, 30, 7, 7])
label torch.Size([2, 25, 7, 7])


In [2]:
# from torch import Tensor
# from torch.nn import Module

# class MeanAvgPrecision(Module):
#     def __init__(self):
#         super().__init__()
        
#     def forward(self, output, label):
#         classes = 20
#         bboxes = (output.shape[1] - classes) // 5
#         assert((output.shape[1] - classes) % 5 == 0)
        
#         index_table = self.generate_cell_index_table(output)
#         output_copy = output.clone()
        
#         for c in range(20):
            
            
#             class_mask = torch.eq(torch.argmax(output[:, -classes:, :, :], 1, keepdim=True), c)
            
#             # [2, 10, 49], [2, 10, 0], ... [B, 5 * 2, I]
#             class_bboxes = torch.masked_select(output[:, :bboxes * 5, :, :], class_mask).view(output.shape[0], 10, -1)
#             # [2, 2, 49], [2, 2, 0], ... [B, 2, I]
#             class_bbox_indicies = torch.masked_select(index_table, class_mask).view(output.shape[0], 2, -1)
            
#             if class_bboxes.shape[2] == 0:
#                 continue
            
#             print(class_bboxes[0, :, 42])
#             class_bboxes[0, 0, 42] = 0.3
#             print(class_bboxes[0, :, 42])
            
#             # concat two box matrix
#             ##! TODO: idx_y, idx_x OR idx_x, idx_y ?
#             # [B, 7, I], 7 -> [x, y, w, h, c, idx_y, idx_x]
#             class_bboxes = class_bboxes.view(class_bboxes.shape[0], 5, -1)
#             class_bbox_indicies = torch.repeat_interleave(class_bbox_indicies, 2, 1).view(class_bbox_indicies.shape[0], 2, -1)
#             class_bboxes_with_index = torch.cat([class_bboxes, class_bbox_indicies], dim=1)
#             class_bbox_xyminmax = self.xywh_to_xyminmax(class_bboxes_with_index)  # [B, 5, I] -> xmin, ymin, xmax, ymax, confidence
            
#             # print(class_bbox_xyminmax == )
            
# #             if class_bbox_xyminmax.shape[2] > 2:
# #                 for i in range(class_bbox_xyminmax.shape[2]):
# #                     print(class_bbox_xyminmax[0, :, i])
            
#             confidence_indicies = torch.argsort(class_bboxes_with_index[:, 4:5], dim=0, descending=True).repeat(1, 7, 1)
            
#             # [B, 7, 98] sorted by confidence in descending order
#             sorted_class_bboxes_with_index = torch.gather(class_bboxes_with_index, 1, confidence_indicies)
            
# #             # Iterate over "sorted" boxes
# #             # first box would have largest confidence value!
# #             # current_bbox -> [B, 7], 7 -> [x, y, w, h, c, idx_y, idx_x]
# #             for mbidx in range(0, sorted_class_bboxes_with_index.shape[2]):
# #                 if sorted_class_bboxes_with_index[:, 4, mbidx] == 0:
# #                     continue
                    
# #                 max_bbox = class_bboxes_with_index[:, :, mbidx]
                
# #                 # get iou between first element and another bboxes
                
                
# #                 # sorted_class_bboxes_with_index[iou > 0.5, b] = 0
                
                
# # #             print("Class %d entries: " % c, class_bboxes.shape)
# # #             print("Class %d entry indicies must be [B, 2, I] and same B, I as above: \n\t" % c, class_bbox_indicies.shape)
                
#         # compare current output tensor with copied original tensor
#         # with element-wise!
#         print(torch.all(output == output_copy))
        
#         return 0
    
#     @staticmethod
#     def xywh_to_xyminmax(xywh, cell_size = 7):
#         # xywh -> [B, 7, I], 7 -> [x, y, w, h, c, y, x]
#         cx = (xywh[:, 0, :] + xywh[:, 6, :]) / cell_size
#         cy = (xywh[:, 1, :] + xywh[:, 5, :]) / cell_size
        
#         xmin, xmax = torch.maximum(cx - (xywh[:, 2, :] / 2), torch.Tensor([0])), torch.minimum(cx + (xywh[:, 2, :] / 2), torch.Tensor([1]))
#         ymin, ymax = torch.maximum(cy - (xywh[:, 3, :] / 2), torch.Tensor([0])), torch.minimum(cy + (xywh[:, 3, :] / 2), torch.Tensor([1]))
                
#         return torch.stack([xmin, ymin, xmax, ymax, xywh[:, 4, :]], dim=1)
        
    
#     @staticmethod
#     def generate_cell_index_table(output):
#         index_map_x = torch.arange(0, 7, device=output.device).repeat(7)
#         index_map_y = torch.repeat_interleave(torch.arange(0, 7, device=output.device), 7)
#         index_map = torch.unsqueeze(torch.stack([index_map_y, index_map_x], dim=0).view(2, 7, 7), 0)
#         return index_map
#         # index_map -> [1, 2, 7, 7]
    
#     @staticmethod
#     def get_iou_xywh(input_xywh: Tensor, label_xywh: Tensor) -> Tensor:
#         index_map_x = torch.arange(0, 7, device=input_xywh.device).repeat(7)
#         index_map_y = torch.repeat_interleave(torch.arange(0, 7, device=input_xywh.device), 7)
#         index_map = torch.unsqueeze(torch.stack([index_map_y, index_map_x], dim=0).view(2, 7, 7), 0)
#         input_xy_global = (input_xywh[:, :2, :, :] + index_map) / 7
#         input_width_half, input_height_half = (input_xywh[:, 2, :, :] / 2), (input_xywh[:, 3, :, :] / 2)
#         input_xmin = input_xy_global[:, 0, :, :] - input_width_half  # x_center - width / 2
#         input_xmax = input_xy_global[:, 0, :, :] + input_width_half
#         input_ymin = input_xy_global[:, 1, :, :] - input_height_half
#         input_ymax = input_xy_global[:, 1, :, :] + input_height_half

#         label_xy_global = (label_xywh[:, :2, :, :] + index_map) / 7
#         label_width_half, label_height_half = (label_xywh[:, 2, :, :] / 2), (label_xywh[:, 3, :, :] / 2)
#         label_xmin = label_xy_global[:, 0, :, :] - label_width_half  # x_center - width / 2
#         label_xmax = label_xy_global[:, 0, :, :] + label_width_half
#         label_ymin = label_xy_global[:, 1, :, :] - label_height_half
#         label_ymax = label_xy_global[:, 1, :, :] + label_height_half

#         input_volume = input_xywh[:, 2, :, :] * input_xywh[:, 3, :, :]
#         label_volume = label_xywh[:, 2, :, :] * label_xywh[:, 3, :, :]
#         intersect_width = torch.minimum(input_xmax, label_xmax) - torch.maximum(input_xmin, label_xmin)
#         intersect_height = torch.minimum(input_ymax, label_ymax) - torch.maximum(input_ymin, label_ymin)
#         intersect_volume = intersect_width * intersect_height
#         union_volume = input_volume + label_volume - intersect_volume

#         return intersect_volume / union_volume
        
# mean_avg_precision = MeanAvgPrecision()
# print(mean_avg_precision(output, label))

In [28]:
from torch import Tensor
from torch.nn import Module

class MeanAvgPrecision(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, output, label):
        classes = 20
        bboxes = (output.shape[1] - classes) // 5
        assert((output.shape[1] - classes) % 5 == 0)
        
        # Make every predictor+class set flatten
        # output [B * (5+5+20) * C * C] -> bbox_scores [B * (5+20) * (C^2 * 2)]
        index_table = self.generate_cell_index_table_order_xy(output)
        bbox_scores = []
        for b in range(bboxes):
            bbox_xywhc = output[:, b * 5:(b + 1) * 5, :, :]
            class_scores = output[:, -classes:, :, :]
            bbox_xyminmaxc = self.xywhc_to_xyminmaxc(bbox_xywhc, index_table)  # 2, 5, I
            bbox_scores.append(torch.cat([bbox_xyminmaxc, class_scores.view(class_scores.shape[0], classes, -1)], dim=1))
        bbox_scores = torch.cat(bbox_scores, dim=2)  # -> torch.Size([2, 25, 98])
        bbox_scores[:, -20:, :][bbox_scores[:, -20:, :] < 0.2] = 0
        predictors = bbox_scores.shape[2]
        
        # Iterate over classes and do NMS algorithm
        for c in range(classes):
            # x, y, w, h, c, p(classprob)
            current_xywhc_slice = bbox_scores[:, :5, :]
            current_class_score_slice = bbox_scores[:, c+5:c+6, :]
            
            current_xywhcp_desc_indicies = torch.argsort(current_class_score_slice, dim=1)
            for p in range(predictors):
                current_predictor_xywhc = current_xywhc_slice[: , :, p]
                current_predictor_class_score = current_class_score_slice[:, :, p]
                
                print(current_predictor_xywhc.shape)
                print(current_predictor_class_score.shape)
                ious = self.get_iou_between(current_xywhcp, )
        
        return 0
    
    @staticmethod
    def xywhc_to_xyminmaxc(xywhc, index_table):  # using index_table with args minimizing unnecessary overhead
        # xywhc -> [B, 5, C, C]
        cell_size = xywhc.shape[2]
        cxcy = (xywhc[:, :2, :, :] + index_table) / cell_size
        
        xmin = torch.maximum(cxcy[:, 0, :, :] - (xywhc[:, 2, :, :] / 2), torch.Tensor([0]))
        xmax = torch.minimum(cxcy[:, 0, :, :] + (xywhc[:, 2, :, :] / 2), torch.Tensor([1]))
        ymin = torch.maximum(cxcy[:, 1, :, :] - (xywhc[:, 3, :, :] / 2), torch.Tensor([0]))
        ymax = torch.minimum(cxcy[:, 1, :, :] + (xywhc[:, 3, :, :] / 2), torch.Tensor([1]))
        return torch.stack([xmin, ymin, xmax, ymax, xywhc[:, 4, :, :]], dim=1).view(xywhc.shape[0], 5, -1)
        
    
    @staticmethod
    def generate_cell_index_table_order_xy(output):
        index_map_x = torch.arange(0, 7, device=output.device).repeat(7)
        index_map_y = torch.repeat_interleave(torch.arange(0, 7, device=output.device), 7)
        index_map = torch.unsqueeze(torch.stack([index_map_x, index_map_y], dim=0).view(2, 7, 7), 0)
        return index_map
        # index_map -> [1, 2, 7, 7]
        
    @staticmethod
    def get_iou_between(xywhcp1: Tensor, xywhcp2: Tensor) -> Tensor:
        
        return None
    
    @staticmethod
    def get_iou_xywh(input_xywh: Tensor, label_xywh: Tensor) -> Tensor:
        index_map_x = torch.arange(0, 7, device=input_xywh.device).repeat(7)
        index_map_y = torch.repeat_interleave(torch.arange(0, 7, device=input_xywh.device), 7)
        index_map = torch.unsqueeze(torch.stack([index_map_y, index_map_x], dim=0).view(2, 7, 7), 0)
        input_xy_global = (input_xywh[:, :2, :, :] + index_map) / 7
        input_width_half, input_height_half = (input_xywh[:, 2, :, :] / 2), (input_xywh[:, 3, :, :] / 2)
        input_xmin = input_xy_global[:, 0, :, :] - input_width_half  # x_center - width / 2
        input_xmax = input_xy_global[:, 0, :, :] + input_width_half
        input_ymin = input_xy_global[:, 1, :, :] - input_height_half
        input_ymax = input_xy_global[:, 1, :, :] + input_height_half

        label_xy_global = (label_xywh[:, :2, :, :] + index_map) / 7
        label_width_half, label_height_half = (label_xywh[:, 2, :, :] / 2), (label_xywh[:, 3, :, :] / 2)
        label_xmin = label_xy_global[:, 0, :, :] - label_width_half  # x_center - width / 2
        label_xmax = label_xy_global[:, 0, :, :] + label_width_half
        label_ymin = label_xy_global[:, 1, :, :] - label_height_half
        label_ymax = label_xy_global[:, 1, :, :] + label_height_half

        input_volume = input_xywh[:, 2, :, :] * input_xywh[:, 3, :, :]
        label_volume = label_xywh[:, 2, :, :] * label_xywh[:, 3, :, :]
        intersect_width = torch.minimum(input_xmax, label_xmax) - torch.maximum(input_xmin, label_xmin)
        intersect_height = torch.minimum(input_ymax, label_ymax) - torch.maximum(input_ymin, label_ymin)
        intersect_volume = intersect_width * intersect_height
        union_volume = input_volume + label_volume - intersect_volume

        return intersect_volume / union_volume
        
mean_avg_precision = MeanAvgPrecision()
print(mean_avg_precision(output, label))

torch.Size([2, 5])
torch.Size([2, 1])


NameError: name 'current_xywhcp' is not defined

In [4]:
torch.repeat_interleave(torch.Tensor([[1], [2], [3]]), 4, 1)

tensor([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.]])